Installing required packages

In [ ]:
!pip install flask google-cloud-storage streamlit pyngrok paho-mqtt calplot twilio statsmodels pandas pmdarima sklearn

Autenticating Google Cloud account

In [3]:
from google.colab import auth

auth.authenticate_user()
print("Authenticated")

Authenticated


Starting the Flask server

In [4]:
from flask import Flask, Response
from google.cloud import storage
from threading import Timer
import os

app = Flask(__name__)

bucket_name = 'BUCKET_NAME'

# Get feeding times csv file
@app.route('/get_feed_csv')
def get_feed_csv():
    # Set up Google Cloud Storage client
    storage_client = storage.Client()
    file_name = 'data_feed.csv'

    # Read the CSV file from the bucket
    bucket = storage_client.get_bucket(bucket_name)
    blob = storage.Blob(file_name, bucket)
    csv_content = blob.download_as_text()

    # Serve the CSV file
    return Response(csv_content, content_type='text/csv')

# Get empty dispenser times csv file
@app.route('/get_laser_csv')
def get_laser_csv():
    # Set up Google Cloud Storage client
    storage_client = storage.Client()
    file_name = 'data_laser.csv'

    # Read the CSV file from the bucket
    bucket = storage_client.get_bucket(bucket_name)
    blob = storage.Blob(file_name, bucket)
    csv_content = blob.download_as_text()

    # Serve the CSV file
    return Response(csv_content, content_type='text/csv')

# Get latest image
@app.route('/get_latest_image')
def get_latest_image():
    # Get the blobs in the bucket
    storage_client = storage.Client()
    blobs = storage_client.list_blobs(bucket_name)

    # Filter blobs to only get image files
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif'}
    image_blobs = [blob for blob in blobs if os.path.splitext(blob.name)[1] in image_extensions]

    # If no image files were found, return an error
    if not image_blobs:
        return Response("No image files found in bucket", status=404)

    # Find the latest image file
    latest_blob = max(image_blobs, key=lambda blob: blob.time_created)

    # Download the blob as bytes
    image_bytes = latest_blob.download_as_bytes()

    # Serve the image
    return Response(image_bytes, mimetype=latest_blob.content_type)

def run_flask():
    app.run(port=5000)

# Run the Flask API in a separate thread
flask_thread = Timer(1, run_flask)
flask_thread.start()


Streamlit script

In [5]:
app_script = """
import streamlit as st
import pandas as pd
import requests
from io import StringIO
import plotly.express as px
import paho.mqtt.client as mqtt
import base64
import io
from PIL import Image
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm

### MQTT

# Create an MQTT client
client = mqtt.Client()

# Set the username and password (Replace 'ADAFRUIT_USERNAME' and 'ADAFRUIT_KEY' with your actual username and key)
client.username_pw_set('ADAFRUIT_USERNAME', 'ADAFRUIT_KEY')

# Connect to the Adafruit IO MQTT broker
client.connect('io.adafruit.com', 1883)

### SLIDERS

# Display a slider to set the daily limit
daily_limit = st.sidebar.slider("Daily feeding limit", 1, 10, 3)
st.sidebar.write(f"Daily feeding limit set to: {daily_limit} times")

# Publish the daily limit to the 'daily_limit' feed. Replace 'ADAFRUIT_USERNAME' with your actual username
client.publish('ADAFRUIT_USERNAME/feeds/daily-limit', str(daily_limit))

# Display a slider to set the dispensing interval (in minutes)
dispense_interval = st.sidebar.slider("Minimum dispensing interval", 1, 300, 5)
st.sidebar.write(f"Minimum dispensing interval set to: {dispense_interval} minutes")

# Publish the dispensing interval to the 'dispense_interval' feed. Replace 'ADAFRUIT_USERNAME' with your actual username
client.publish('ADAFRUIT_USERNAME/feeds/min-dispense-interval', str(dispense_interval))

# FETCH AND PLOT DATA

# Function to fetch csv
def fetch_csv(flask_api_url):
    response = requests.get(flask_api_url)
    csv_content = response.text

    # Read the CSV data into a Pandas DataFrame
    csv_file = StringIO(csv_content)
    df = pd.read_csv(csv_file, sep=',') 

    # Convert the timestamp column to a datetime object
    df['time'] = pd.to_datetime(df['time'])

    return df

# Function to print table and plot the data
def plot_data(df, title):
    # Display the data in a table
    st.title(title)
    st.dataframe(df['time'])

    # Extract the date and hour from the timestamp
    df['date'] = df['time'].dt.date
    df['hour'] = df['time'].dt.hour

    # Create a scatter plot of the events
    fig = px.scatter(df, x='date', y='hour')

    # Display the plot in Streamlit
    st.plotly_chart(fig)


# Fetch the CSV feeding data from the Flask API and plot the data
df_feed = fetch_csv("http://localhost:5000/get_feed_csv")
plot_data(df_feed, "Dog Feeding Times")

# Fetch the CSV laser data from the Flask API and plot the data
df_laser = fetch_csv("http://localhost:5000/get_laser_csv")
plot_data(df_laser, "Empty Dispenser Times")

### IMAGE 


flask_api_image_url = "http://localhost:5000/get_latest_image"

# Function to fetch the latest image from the Google Cloud Storage
def fetch_image():
    response = requests.get(flask_api_image_url)
    image_bytes = response.content

    # Convert the bytes to a PIL Image
    image = Image.open(io.BytesIO(image_bytes))

    # Display the image in Streamlit
    st.image(image, caption='Latest Image')

# Fetch image when app starts
st.title("Latest Image")
fetch_image()

# Button to fetch latest image
if st.button('Refresh Image'):
    st.experimental_rerun()

### PREDICTIONS

# Predictions
st.title("Predictions")
st.write("computing... This could take a while")

# Function to predict the next event times
def predict_next_event(df, title, m_value):
  try:
    # Compute time difference between events in hours
    df['time_diff'] = df['time'].diff().dt.total_seconds() / 3600

    # Drop the first row which has a NaN value for 'time_diff'
    df = df.dropna()

    # Set 'time' as the index
    df = df.set_index('time')

    # Drop the first row, which will have a NaN value for time_diff
    df = df.iloc[1:]

    # Run the model
    model = pm.auto_arima(df['time_diff'], seasonal=True, m=m_value, max_p=5, max_d=5,max_q=5, max_P=5, max_D=5,max_Q=5)
    forecast = model.predict(n_periods=1)
    next_event_time = df.index[-1] + pd.Timedelta(hours=float(forecast.item()))

    # Print the prediction
    st.write(f"Predicted next {title} time: {next_event_time}")

  except (ValueError, IndexError) as e:
    st.write(f"Not enough data points for predicting next {title} time . Please gather more data.")



# Predict next feeding time
predict_next_event(df_feed, "feeding", 4)

# Predict next empty dispenser time
predict_next_event(df_laser, "empty dispenser", 1)


"""

with open("streamlit_app.py", "w") as f:
    f.write(app_script)


Ngrok auth token

In [8]:
!ngrok authtoken AUTH_TOKEN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Launching webapp

In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print(f"Streamlit app running at {public_url}")
!streamlit run --server.port 8501 streamlit_app.py

Streamlit app running at NgrokTunnel: "https://4690-35-199-161-15.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Network URL: http://100.12.0.12:8501
  External URL: http://40.187.130.11:8501



Launching webapp with SMS 

In [ ]:
from twilio.rest import Client
from pyngrok import ngrok

account_sid = 'ACCOUNT_SID'
auth_token = 'AUTH_TOKEN'
twilio_client = Client(account_sid, auth_token)

public_url = ngrok.connect(8501)

message = twilio_client.messages.create(
    body=f'Streamlit app running at {public_url}',
    from_='TWILIO_NUMBER',
    to='YOUR_NUMBER'
)

print(f"Streamlit app running at {public_url}")
!streamlit run --server.port 8501 streamlit_app.py


Stop all ngrok processes

In [ ]:
ngrok.kill()